<a href="https://colab.research.google.com/github/Nayana278820/Hangman/blob/master/MP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MP5

# Practice using text embeddings for search and for classification

## store in a vector database

## find nearest neighbors

## classify by repo typy and by repo

## To use upload/download you need to login into HGF CLI; use write token created on HGF

In [2]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `MP5T` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/hu

In [3]:
!huggingface-cli download --repo-type dataset --local-dir . fdac24/MP4 --include "*.json.gz"
!huggingface-cli download --repo-type dataset --local-dir . fdac24/MP3 --include "*.json.gz"

Fetching 93 files:   0% 0/93 [00:00<?, ?it/s]Downloading 'alecfowl_embed.json.gz' to '.cache/huggingface/download/alecfowl_embed.json.gz.f8eee4d10f784c094d002b7ca18dc9093ae5bf020838c463a3fc6999f455e2ec.incomplete'

alecfowl_embed.json.gz:   0% 0.00/17.2M [00:00<?, ?B/s]

audris_embed.json.gz:   0% 0.00/108k [00:00<?, ?B/s]


amcclu13_embed.json.gz:   0% 0.00/17.1M [00:00<?, ?B/s]



aking100_embed.json.gz:   0% 0.00/18.5M [00:00<?, ?B/s]




aroman_embed.json.gz:   0% 0.00/16.9M [00:00<?, ?B/s]





audris_embed.json.gz: 100% 108k/108k [00:00<00:00, 4.31MB/s]
Download complete. Moving file to audris_embed.json.gz


amuell11_embed.json.gz:   0% 0.00/1.23M [00:00<?, ?B/s]






awarden9_embed.json.gz:   0% 0.00/17.4M [00:00<?, ?B/s]Downloading 'aweis3_embed.json.gz' to '.cache/huggingface/download/aweis3_embed.json.gz.7168c23bd0e7f5cff58b70842cce768b2e6e111e1bc0b902da04f52b399c76c6.incomplete'
amuell11_embed.json.gz: 100% 1.23M/1.23M [00:00<00:00, 14.0MB/s]
Download complete. Moving file

# Now read in embeddings

In [4]:
import json, re, gzip
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import json, re, gzip

utids4 = !ls *_embed.json.gz|sed 's|_embed.json.gz||'
labels = []
X = []
for utid in (utids4):
  try:
        fi = gzip.open(f"{utid}_embed.json.gz", 'r')
        for line in fi:
            line = line.strip()
            #(line)
            if len(line) < 10:
                continue  # Skip short lines
            try:
                id, typ, section, vector = json.loads(line)
                labels.append({'id':id, 'type':typ, 'section':section, 'utid':utid, 'emb':np.array(vector)})
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error processing line in {utid}_embed.json.gz: {e}")
                print(f"Problematic line: {line}")
                # continue  # Skip to the next line
                # break  # Exit the inner loop
  except EOFError:
        print(f"EOFError encountered while reading {utid}_embed.json.gz. Skipping this file.")
        continue  # Skip to the next utid

# LOOK FOR ANOMALIES
check = pd.crosstab([l['utid'] for l in labels], [l['section'] == '0: Preamble' for l in labels])
print(check)


EOFError encountered while reading edayney_embed.json.gz. Skipping this file.
col_0     False  True 
row_0                 
aking100   2729    202
alecfowl   2435    180
amarlow6   1280    106
amcclu13   2379    185
amuell11      0    173
...         ...    ...
wwinslad   2465    192
yarddoga   2740    171
ygaikwad      0    179
yhg461     2403    181
zyr546     2339    185

[89 rows x 2 columns]


# instances with a single section are problematic: exclude

In [5]:
df = pd.DataFrame(labels)
utidsG = []
dfG = pd.DataFrame()
Xg = []
for utid in (utids4):
  df1 = df[(df['utid']==utid) & (df['section'] != '0: Preamble')]
  if len(df1) > 0:
    utidsG.append(utid)
    dfG = pd.concat([dfG, df[df['utid']==utid] ], ignore_index=True)


# Now use embedding to find most similar sections

In [6]:
# this takes forever, use vector database instead
def csim(v1, v2):
    dot_product = sum(a * b for a, b in zip(v1, v2))
    magnitude_v1 = sum(a * a for a in v1) ** 0.5
    magnitude_v2 = sum(b * b for b in v2) ** 0.5
    return dot_product / (magnitude_v1 * magnitude_v2)

def mostSim (i, dfMy):
  mx = -1
  mxi = -1
  # for j in range(len(dfG)):
  #   if(i != j):
  #     v = csim(dfMy.iloc[i]['emb'], dfG.iloc[j]['emb'])
  #     if v > mx:
  #       mx = v
  #       mxi = j
  # return (mxi,mx,dfG[mxi])

closest = []
# find sections closest to the sections you have extracted
mynetid = 'spatil12'
# Check if 'utid' column exists in dfG and if mynetid is present
if 'utid' in dfG.columns and mynetid in dfG['utid'].values:
    dfMy = dfG[dfG['utid']==mynetid]
    for i in range(len(dfMy)):
        closest.append(mostSim(i, dfMy))
else:
    print(f"Warning: Either 'utid' column is missing or '{mynetid}' is not found in the 'utid' column of dfG.")


## install vector database

In [7]:
! pip install pymilvus milvus --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.1 MB/s eta 0:00:00


In [8]:
from milvus import default_server
from pymilvus import connections, utility
default_server.start()
connections.connect(host="127.0.0.1", port=default_server.listen_port)

In [9]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

COLLECTION_NAME = "hfh"
DIMENSION = 768
BATCH_SIZE = 128
TOPK = 3
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="repo", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="section", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="type", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="utid", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)
collection = Collection(name=COLLECTION_NAME, schema=schema)

index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128},
}
collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

In [10]:
for i in range(len(dfG)):
   d = dfG.iloc[i]
   #df1 = df[(df['ID']== id) & (df['section'] == section)]['content']
   #if (len(df1) > 1):    df1 = df1.iloc[0]
   #else:    if (len(df1) < 10): continue
   #cnt = df1
   section = d['section']
   lenT = len(section)
   if lenT > 30: section = section[:30]
   ins =  [ [ d['id'] ], [ section ], [ d['type'] ], [ d['utid'] ], [ d['emb' ] ] ]
   collection.insert(ins)

In [11]:
neighbors = []
for i in range(len(dfG)):
  v = collection.search (
    data= [ dfG.iloc[i]['emb'] ],  # Embeded search value
    anns_field="embedding",        # Search across embeddings
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=TOPK,  # Return 3 nearest neighbours
    output_fields=[ "repo", "type", "section", 'utid' ])
  neighbors .append(v)


# See if the closest neighor has the same repo, utid, or type

In [12]:

tot = 0
mmatch = 0
for i in range(len(dfMy)):
  if dfG.iloc[i]['id'] ==neighbors[i][0][0].entity.get("repo"):
      mmatch+=1
  tot += 1
print(mmatch, tot, mmatch/tot)
umatch = 0
for i in range(len(dfMy)):
  if dfG.iloc[i]['utid'] ==neighbors[i][0][0].entity.get("utid"):
      umatch+=1
print(umatch, tot, umatch/tot)
tmatch = 0
for i in range(len(dfMy)):
  if dfG.iloc[i]['type'] ==neighbors[i][0][0].entity.get("type"):
      tmatch+=1
print(tmatch, tot, umatch/tot)

591 2714 0.21775976418570375
602 2714 0.22181282240235814
2393 2714 0.22181282240235814


# Create a classifier based on embeddings

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Regression needs separate columns
dfO = dfG[dfG['utid']!='utid']
X = np.array(dfO['emb'])
Xh = np.vstack(X)
X = np.array(dfMy['emb'])
Xmy = np.vstack(X)

rtype = [ (l=='model')+1 for l in dfG['type']]
# try type
X_train, X_test, y_train, y_test = train_test_split(Xh, rtype, test_size=0.2)
#print(X_train)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
#on test set
accuracy = classifier.score(X_test, y_test)
print("Accuracy for type:", accuracy)
# on validation set
accuracy = classifier.score(Xmy, [ (l=='model')+1 for l in dfMy['type']])
print("Accuracy on validation set:", accuracy)

Accuracy for type: 0.9133015977237907
Accuracy on validation set: 0.9347826086956522


In [14]:
classifier.fit(X_train[:,:10], y_train)
accuracy = classifier.score(X_test[:,:10], y_test)
print("Accuracy for type (10 predictors):", accuracy)

Accuracy for type (10 predictors): 0.7923232654847888


In [15]:
from sklearn.svm import SVC
clsvm = SVC(kernel='linear')
clsvm = clsvm.fit(X_train[:,:10], y_train)
accuracy = clsvm.score(X_test[:,:10], y_test)
print("Accuracy for type (svm, 10 predictors):", accuracy)

Accuracy for type (svm, 10 predictors): 0.8073703217334208
